<a href="https://colab.research.google.com/github/s-c-soma/AdvanceDeeplearning-CMPE-297/blob/master/Assignment_5/Assignment_5_Text_Classification_with_Transformer_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text classification with Transformer




## **Implementation Details and Discussion:**

* In this assignment, I have implemented a transformer block called Multi Head Self attention. 
* Code is implemented in Keras. I have followed Professor's reference links and typed everything from scrath here.
* Here I have used IMDB text dataset. I am considering the top 20k words
* The transformer implemented here is used for text classification.
* Two seperate embedding layers is developed, one for tokens, one for token index (positions).
* Transformer layer outputs one vector for each time step of our input sequence.
* Here, I take the mean across all time steps and use a feed forward network on top of it to classify text.
* After 20 epochs, result is as follows:
> loss: 0.0045 - accuracy: 0.9984 - val_loss: 1.6841 - val_accuracy: 0.8248

## Setup- Packeges


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


## Implementing: Multi Head self Attention Layer


In [2]:

class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output



## Implementing: Transformer Block


In [3]:

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)



## Implementing: Embedding Layer




In [4]:

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions



## Load Data: IMDB


In [5]:
vocab_size = 20000  # 
maxlen = 200  # Only consider the first 200 words of each movie review
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)


17465344/17464789 [==============================] - 1s 0us/step
25000 Training sequences
25000 Validation sequences


## Classifier Model using Transformer Layer




In [6]:

embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)



## Training


In [7]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

## Evaluation

In [9]:

history = model.fit(
    x_train, y_train, batch_size=32, epochs=20, validation_data=(x_val, y_val)
)


Epoch 1/20
782/782 [==============================] - 11s 14ms/step - loss: 0.0150 - accuracy: 0.9956 - val_loss: 0.9560 - val_accuracy: 0.8310
Epoch 2/20
782/782 [==============================] - 11s 14ms/step - loss: 0.0128 - accuracy: 0.9963 - val_loss: 1.1160 - val_accuracy: 0.8323
Epoch 3/20
782/782 [==============================] - 11s 14ms/step - loss: 0.0123 - accuracy: 0.9960 - val_loss: 1.2048 - val_accuracy: 0.8308
Epoch 4/20
782/782 [==============================] - 11s 14ms/step - loss: 0.0125 - accuracy: 0.9959 - val_loss: 1.0902 - val_accuracy: 0.8320
Epoch 5/20
782/782 [==============================] - 11s 14ms/step - loss: 0.0087 - accuracy: 0.9973 - val_loss: 1.1075 - val_accuracy: 0.8326
Epoch 6/20
782/782 [==============================] - 11s 14ms/step - loss: 0.0094 - accuracy: 0.9970 - val_loss: 1.2374 - val_accuracy: 0.8208
Epoch 7/20
782/782 [==============================] - 11s 14ms/step - loss: 0.0068 - accuracy: 0.9978 - val_loss: 1.3188 - val_accuracy: